In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('PolitifactDatasetFeatures.csv')

In [3]:
df

,Unnamed: 0.1,Unnamed: 0,Statement Author,Statement,Rating,cleaned,statement_length,word_count,sentence_count,unique_words,...,punctuation_count,noun_%,pronoun_%,verb_%,adj_%,determiner_%,foreign_%,cleaned_tokenized,emotion,profanity
0,0,0,Fentrice Driskell,"""$1 of every $3 (Ron DeSantis) spends comes fr...",true,every ron desantis spends come federal government,73,49,1,12,...,7,0.157895,0.000000,0.000000,0.052632,0.105263,0.0,"['every', 'ron', 'desantis', 'spends', 'come',...",Happy,0.008152
1,1,1,Robert Ortt,If New York’s proposed limits on natural gas i...,true,new york proposed limit natural gas building t...,127,98,1,19,...,2,0.407407,0.000000,0.185185,0.111111,0.074074,0.0,"['new', 'york', 'proposed', 'limit', 'natural'...",Surprise,0.056929
2,2,2,Tony Evers,“Wisconsin is the nation’s top cranberry produ...,true,wisconsin nation top cranberry producer fact f...,122,87,1,18,...,2,0.428571,0.035714,0.071429,0.071429,0.071429,0.0,"['wisconsin', 'nation', 'top', 'cranberry', 'p...",Surprise,0.039250
3,3,3,Morgan Luttrell,"""Biden drained America's Strategic Petroleum R...",true,biden drained america strategic petroleum rese...,86,68,1,12,...,4,0.312500,0.000000,0.062500,0.000000,0.062500,0.0,"['biden', 'drained', 'america', 'strategic', '...",Fear,0.030024
4,4,4,Melissa Agard,"""Historically, our spring elections (including...",true,historically spring election including state s...,117,85,1,16,...,6,0.272727,0.090909,0.136364,0.000000,0.045455,0.0,"['historically', 'spring', 'election', 'includ...",Surprise,0.008631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,1135,1135,TikTok posts,"“As of today, no one has the right to film or ...",pants-fire,today one right film photograph mr biden climb...,138,91,3,22,...,6,0.454545,0.030303,0.030303,0.000000,0.090909,0.0,"['today', 'one', 'right', 'film', 'photograph'...",Fear,0.121310
1136,1136,1136,Facebook posts,"""mRNA is not a vaccine"" — it's ""actually an op...",pants-fire,mrna vaccine actually operating system run bil...,105,66,1,19,...,7,0.307692,0.038462,0.038462,0.000000,0.076923,0.0,"['mrna', 'vaccine', 'actually', 'operating', '...",Fear,0.049405
1137,1137,1137,Facebook posts,Video says COVID-19 vaccines are “weapons of m...,pants-fire,video say covid vaccine weapon mass destructio...,100,69,1,16,...,2,0.380952,0.000000,0.142857,0.095238,0.047619,0.0,"['video', 'say', 'covid', 'vaccine', 'weapon',...",Fear,0.005492
1138,1138,1138,Facebook posts,Says Ben Shapiro said on Twitter that his “red...,pants-fire,say ben shapiro said twitter red pill moment s...,96,69,1,17,...,1,0.450000,0.050000,0.200000,0.050000,0.000000,0.0,"['say', 'ben', 'shapiro', 'said', 'twitter', '...",Happy,0.297600


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import metrics
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

from sklearn.metrics import cohen_kappa_score

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [5]:
import nltk
import nltk.stem as ns
import string
import re

ps = ns.PorterStemmer()
lemma = ns.WordNetLemmatizer()

from nltk.tokenize import word_tokenize, sent_tokenize
def remove_punctuation(x):
    punctuation = string.punctuation
    no_punct = "".join([word for word in x if word not in punctuation])
    return no_punct

def remove_stopwords(x):
    stopwords = nltk.corpus.stopwords.words('english')
    no_sw = [word for word in x if word not in stopwords]
    return no_sw

#function built to use either stemming or lematization
def lemmatize(x):
    lemmatized = [lemma.lemmatize(word) for word in x]
    return lemmatized


#all of those functions inside one function to keep code clean
def clean_data(x):
    #tokens = re.sub("[^a-zA-Z]", " ", x.lower())
    essay_v = re.sub("[^a-zA-Z]", " ", x)
    tokens = essay_v.lower().split()
    no_sw = remove_stopwords(tokens)
    root = lemmatize(no_sw)
    cleaned = ' '.join(root)
    return cleaned

def clean_tokenize(x):
    #tokens = re.sub("[^a-zA-Z]", " ", x.lower())
    essay_v = re.sub("[^a-zA-Z]", " ", x)
    tokens = essay_v.lower().split()
    no_sw = remove_stopwords(tokens)
    root = lemmatize(no_sw)
    return root


In [6]:
vectorizer = TfidfVectorizer(analyzer=clean_tokenize)
feat_df = df[['statement_length', 'word_count', 'sentence_count', 'unique_words', 'lexical_richness', 'punctuation_count', 'noun_%', 'pronoun_%', 'verb_%', 'adj_%', 'determiner_%', 'foreign_%','emotion', 'profanity']]
X = pd.concat([pd.DataFrame(vectorizer.fit_transform(df['Statement']).toarray()), feat_df], axis = 1)
y = df['Rating']

In [7]:
X

,0,1,2,3,4,5,6,7,8,9,...,lexical_richness,punctuation_count,noun_%,pronoun_%,verb_%,adj_%,determiner_%,foreign_%,emotion,profanity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.00,7,0.157895,0.000000,0.000000,0.052632,0.105263,0.0,Happy,0.008152
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,123.48,2,0.407407,0.000000,0.185185,0.111111,0.074074,0.0,Surprise,0.056929
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,101.08,2,0.428571,0.035714,0.071429,0.071429,0.071429,0.0,Surprise,0.039250
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.00,4,0.312500,0.000000,0.062500,0.000000,0.062500,0.0,Fear,0.030024
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.00,6,0.272727,0.090909,0.136364,0.000000,0.045455,0.0,Surprise,0.008631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,102.06,6,0.454545,0.030303,0.030303,0.000000,0.090909,0.0,Fear,0.121310
1136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,112.00,7,0.307692,0.038462,0.038462,0.000000,0.076923,0.0,Fear,0.049405
1137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.00,2,0.380952,0.000000,0.142857,0.095238,0.047619,0.0,Fear,0.005492
1138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.00,1,0.450000,0.050000,0.200000,0.050000,0.000000,0.0,Happy,0.297600


In [8]:
X.columns = X.columns.astype(str)

In [9]:
X

,0,1,2,3,4,5,6,7,8,9,...,lexical_richness,punctuation_count,noun_%,pronoun_%,verb_%,adj_%,determiner_%,foreign_%,emotion,profanity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.00,7,0.157895,0.000000,0.000000,0.052632,0.105263,0.0,Happy,0.008152
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,123.48,2,0.407407,0.000000,0.185185,0.111111,0.074074,0.0,Surprise,0.056929
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,101.08,2,0.428571,0.035714,0.071429,0.071429,0.071429,0.0,Surprise,0.039250
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.00,4,0.312500,0.000000,0.062500,0.000000,0.062500,0.0,Fear,0.030024
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.00,6,0.272727,0.090909,0.136364,0.000000,0.045455,0.0,Surprise,0.008631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,102.06,6,0.454545,0.030303,0.030303,0.000000,0.090909,0.0,Fear,0.121310
1136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,112.00,7,0.307692,0.038462,0.038462,0.000000,0.076923,0.0,Fear,0.049405
1137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.00,2,0.380952,0.000000,0.142857,0.095238,0.047619,0.0,Fear,0.005492
1138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.00,1,0.450000,0.050000,0.200000,0.050000,0.000000,0.0,Happy,0.297600


In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
import joblib

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

xgb = XGBClassifier()

le = LabelEncoder()
y = le.fit_transform(y)

num_cols = ['statement_length', 'word_count', 'sentence_count', 'unique_words', 'lexical_richness', 'punctuation_count']
cat_cols = ['emotion']


ct = ColumnTransformer([('standard_scaler', StandardScaler(), num_cols),
                        ('one_hot_encoder', OneHotEncoder(sparse=False, handle_unknown= "ignore"), cat_cols)], 
                        remainder='passthrough')



In [11]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [25]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, f1_score

clfXGB_nocv= Pipeline(steps = [('preprocessor', ct), ('XGBoost', XGBClassifier())])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=64)

In [26]:
clfXGB_nocv.fit(X_train, y_train)

/Users/carloalmeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standard_scaler',
                                                  StandardScaler(),
                                                  ['statement_length',
                                                   'word_count',
                                                   'sentence_count',
                                                   'unique_words',
                                                   'lexical_richness',
                                                   'punctuation_count']),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['emotion'])])),
                ('XGBoost',
                 XGBClassifier(base_sco...
                               feature_types=None, gamma=0, gpu_id=-1,
                               grow_policy='depthwise', importance_type=None,
                               interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_threshold=64, max_cat_to_onehot=4,
                               max_delta_step=0, max_depth=6, max_leaves=0,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=0, num_parallel_tree=1, predictor='auto',
                               random_state=0, ...))])

In [27]:
clfXGB_nocv.score(X_test, y_test)

0.8114035087719298

In [20]:
#SVM
clfSVM = Pipeline(steps = [('preprocessor', ct), ('SVM', SVC())])
#logistic regression
clfLOGREG = Pipeline(steps = [('preprocessor', ct), ('Logistic Regression', LogisticRegression())])
#XGBoost
clfXGB = Pipeline(steps = [('preprocessor', ct), ('XGBoost', XGBClassifier())])
#Naive Bayes
clfNB = Pipeline(steps = [('preprocessor', ct), ('Naive Bayes', GaussianNB())])
#Random forest
clfRFC = Pipeline(steps = [('preprocessor', ct), ('RFC', RandomForestClassifier())])
#neural netTODO

In [21]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, f1_score, confusion_matrix
from collections import defaultdict
models = [clfSVM, clfLOGREG, clfXGB, clfNB, clfRFC]
fitted_models = []

model_scores=defaultdict(list)

for idx, model in enumerate(models):
    model.fit(X_train, y_train)
    name = list(model.named_steps)[1]
    
    score = model.score(X_test, y_test)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    conf_mat = confusion_matrix(y_test, y_pred)
    tp = conf_mat[0][0]
    tn = conf_mat[1][1]
    fp = conf_mat[0][1]
    fn = conf_mat[1][0]
    precision = tp/(tp+fp)


    
    
    model_scores['Model'].append(name)
    model_scores['Accuracy'].append(score)
    model_scores['MAE'].append(mae)
    model_scores['MSE'].append(mse)
    model_scores['F1 Score'].append(f1)
    model_scores['Confusion Matrix'].append(conf_mat)
    model_scores['Precision'].append(precision)

    fitted_models.append(model)

    

    joblib.dump(model, f'{name}Classifier.joblib')
    print(f'model {idx+1} done')

/Users/carloalmeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


model 1 done
model 2 done


/Users/carloalmeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/carloalmeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


model 3 done
model 4 done


/Users/carloalmeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/carloalmeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


model 5 done


In [22]:
#score, mae, mse, f1, conf_mat, precision
model_scores

defaultdict(list,
            {'Model': ['SVM',
              'Logistic Regression',
              'XGBoost',
              'Naive Bayes',
              'RFC'],
             'Accuracy': [0.793859649122807,
              0.8640350877192983,
              0.8114035087719298,
              0.7675438596491229,
              0.8377192982456141],
             'MAE': [0.20614035087719298,
              0.13596491228070176,
              0.18859649122807018,
              0.2324561403508772,
              0.16228070175438597],
             'MSE': [0.20614035087719298,
              0.13596491228070176,
              0.18859649122807018,
              0.2324561403508772,
              0.16228070175438597],
             'F1 Score': [0.7982832618025751,
              0.8516746411483254,
              0.7922705314009661,
              0.7763713080168776,
              0.8310502283105022],
             'Confusion Matrix': [array([[88, 33],
                     [14, 93]]),
              array([[108,

In [23]:
score_df = pd.DataFrame(model_scores)

In [24]:
score_df

,Model,Accuracy,MAE,MSE,F1 Score,Confusion Matrix,Precision
0,SVM,0.793860,0.206140,0.206140,0.798283,"[[88, 33], [14, 93]]",0.727273
1,Logistic Regression,0.864035,0.135965,0.135965,0.851675,"[[108, 13], [18, 89]]",0.892562
2,XGBoost,0.811404,0.188596,0.188596,0.792271,"[[103, 18], [25, 82]]",0.851240
3,Naive Bayes,0.767544,0.232456,0.232456,0.776371,"[[83, 38], [15, 92]]",0.685950
4,RFC,0.837719,0.162281,0.162281,0.831050,"[[100, 21], [16, 91]]",0.826446
